In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [182]:
#preprocess file
path = "../../logs/"
name = "bd0817_2020-08-19"
raw_file = path + name

with open(raw_file + ".log", 'rb') as file_in:
    with open(raw_file + ".txt", "wb") as file_out:
#         file_out.writelines(filter(lambda line: line.startswith(b'STATE'), file_in))
        file_out.writelines(filter(lambda line: line.startswith(b'STATE'), map(lambda line: line.split(b'|', 3)[3], file_in)))
    
#with open(path + "bd08032_2020-08-05" + ".log", 'rb') as file_in:
#    with open(raw_file + ".txt", "ab") as file_out:
#        file_out.writelines(filter(lambda line: line.startswith(b'STATE'), map(lambda line: line.split(b'|', 3)[3], file_in)))

IndexError: list index out of range

In [181]:
df = pd.read_csv(raw_file + ".txt", sep=":", header=None)
df.columns = ["type", "handid", "stack", "betting", "cards", "result", "player"]

if(len(df.handid)!=len(df.handid.unique())):
    df.handid = df.index
    
df_state = df[df["type"]=="STATE"]
df_state = pd.concat(
    [
        df_state,
        pd.DataFrame(
            [[np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]], 
            index=df_state.index, 
            columns=['stack0','stack1', 'player0','player1', 'result0','result1']
        )
    ], axis=1
)
players = df['player'][0].split("|")
print(players)

EmptyDataError: No columns to parse from file

In [ ]:
s = df_state['stack'].str.split('|')
df_state['stack0'], df_state['stack1'] = s.str[0], s.str[1]
s = df_state['result'].str.split('|')
df_state['result0'], df_state['result1'] = s.str[0], s.str[1]
s = df_state['player'].str.split('|')
df_state['player0'], df_state['player1'] = s.str[0], s.str[1]
df_state[["result0","result1"]] = df_state[["result0","result1"]].apply(pd.to_numeric)

needSwapDataIdx = (df_state['player0'] != players[0])
df_state.loc[needSwapDataIdx,['stack0','stack1']] = df_state.loc[needSwapDataIdx,['stack1','stack0']].values
df_state.loc[needSwapDataIdx,['player0','player1']] = df_state.loc[needSwapDataIdx,['player1','player0']].values
df_state.loc[needSwapDataIdx,['result0','result1']] = df_state.loc[needSwapDataIdx,['result1','result0']].values

df_state['result0_sum'] = df_state['result0'].cumsum()
df_state['result1_sum'] = df_state['result1'].cumsum()

print(len(df_state))
df_state.head()

In [ ]:
#from IPython.display import display, HTML
#display(HTML(df_state.to_html()))

In [ ]:
#Plot of cumulative sum
ax = df_state.plot(x='handid',y=['result0_sum','result1_sum'], figsize=(15,8))
ax.legend(["player0", "player1"]);
ax.title.set_text("%s --- Player0: %s, Player1: %s" % (name, players[0], players[1]))
ax.axhline(y=0, color='red')
plt.show()

print("Player0 Sum: %d" % (df_state[-1:]['result0_sum'].values[0]))
print("Player1 Sum: %d" % (df_state[-1:]['result1_sum'].values[0]))

In [ ]:
player = "seat1"

player_code = df['player'][0].split("|").index(player)
data = df_state['result%d' % player_code].value_counts()

for index, value in data.iteritems():
    if index < 0:
        data[index] = value * -1
        data = data.rename(index={index:(index * -1)})

unique_count = data.index.value_counts()
dummy = unique_count[unique_count == 1].replace(1, 0)
data = data.append(dummy)
data = data.sort_index()

winning = data[data >= 0]
winning = winning.sort_values(ascending = False)
winning = winning[~winning.index.duplicated(keep='first')].sort_index()

losing = data[data<=0]
losing = losing.sort_values(ascending = True)
losing = losing[~losing.index.duplicated(keep='first')].sort_index()

x = data.index.unique()
x_pos = [i for i, _ in enumerate(x)]

fig, ax = plt.subplots(figsize=(15,20))
ax.set_axisbelow(True)
ax.minorticks_on()
ax.grid()

ax.bar(x_pos, winning, color='gold')
ax.bar(x_pos, losing, color='silver')

plt.xticks(x_pos, x, rotation=70)
for index, value in enumerate(winning):
    ax.text(index - 0.3, value + 150, str(value), rotation=90, va='top')
for index, value in enumerate(losing):
    ax.text(index - 0.3, value - 150, str(value), rotation=90)

plt.ylabel("Count")
plt.xlabel("$")
plt.title("%s -- Winning & Losing" % player)

plt.show()